<a href="https://colab.research.google.com/github/rjz46/toxic-interpretability/blob/main/Model_Interpretability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [29]:
!pip install seaborn transformers matplotlib pandas captum lime datasets

# Imports

In [30]:
#standards
import numpy as np
import scipy as sp
import pandas as pd

#systems
import sklearn
import random
import os
import re
import logging
import sys
from tqdm import tqdm
import collections

#plots
import seaborn as sns

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from IPython.display import HTML, display

#sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn import neighbors

from IPython.display import HTML, display

#transformers
from transformers import BertTokenizer

#lime
import lime
from lime.lime_text import LimeTextExplainer


# Load/Process Data

In [31]:
#Load/Parse data
from datasets import load_dataset

#hatexplain dataset
datasets_hate = load_dataset('hatexplain')

#movies dataset
datasets_sentiment = load_dataset('movie_rationales')

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)
Using custom data configuration default
Reusing dataset movie_rationales (/root/.cache/huggingface/datasets/movie_rationales/default/0.1.0/250d4f1e030601f5eaec25ace4fc61ef6cf38508d9a761a72cc7142d3207352d)


In [36]:
print(datasets_hate)
print(datasets_sentiment)

print(datasets_hate['train'][0])
print(datasets_sentiment['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 1924
    })
})
DatasetDict({
    train: Dataset({
        features: ['review', 'label', 'evidences'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['review', 'label', 'evidences'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review', 'label', 'evidences'],
        num_rows: 199
    })
})
{'annotators': {'annotator_id': [203, 204, 233], 'label': [0, 2, 2], 'target': [['Hindu', 'Islam'], ['Hindu', 'Islam'], ['Hindu', 'Islam', 'Other']]}, 'id': '23107796_gab', 'post_tokens': ['u', 'really', 'think', 'i', 'would', 'not', 'have', 'been', 'raped', 'by', 'feral', 'hindu', 'or', '



# Model - BERT

In [34]:

import torch
import torch.nn as nn

from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

datasets = load_dataset('hatexplain')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)


cpu


# Integrated Gradients

# Attention

# Ablation

In [3]:
#BERT Model - 


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')





In [ ]:
#LIME





In [23]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

#Erasure




In [ ]:
#Integrated Gradient


In [ ]:
#Attention


In [ ]:
#

In [ ]:
#Evaluations

In [5]:
import torch
from transformers import AutoTokenizer
dataset = load_dataset('glue', 'mrpc', split='train')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dataset = dataset.map(lambda e: tokenizer(e['sentence1'], truncation=True, padding='max_length'), batched=True)

dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
next(iter(dataloader))

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  7277,  2180,  ...,     0,     0,     0],
         [  101, 10684,  2599,  ...,     0,     0,     0],
         [  101,  1220,  1125,  ...,     0,     0,     0],
         ...,
         [  101,  1109,  2026,  ...,     0,     0,     0],
         [  101, 22263,  1107,  ...,     0,     0,     0],
         [  101,   142,  1813,  ...,     0,     0,     0]]),
 'label': tensor([1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
         1, 1, 0, 0, 1, 1, 1, 0]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])}

In [6]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '<PATH-TO-SAVED-MODEL>'

# load model
model = BertForQuestionAnswering.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

404 Client Error: Not Found for url: https://huggingface.co/%3CPATH-TO-SAVED-MODEL%3E/resolve/main/config.json


OSError: ignored

In [7]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    




# LIME

# Analysis


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [22]:
#Evaluations